In [1]:
import numpy as np
from scipy import stats
#scale 14 @ 
def properties(a):# Standard the physiochemical properties
    temp,r2={},{}
    with open('7_98_hydrophobicity.csv','r') as inpt:
        c=0
        for i in inpt:
            j=i.rstrip().split(',')
            if c==0:
                pass
            else:
                temp[j[0]]=float(j[a])
            c+=1
    r1=stats.zscore(np.array(list(temp.values())))
    for ii,jj in zip(temp.keys(),r1):
        r2[ii]=jj
    return r2
  
no_prop=[r for r in range(1,105)]# 1: Hydrophobicity, 2: Hydrophilicity, 3: mass, 4: pk1, 5:pk2, 6:pi, 20: 14 scale, 60: Tanford
prop={}
for t1 in no_prop:
    prop[t1]=properties(t1)
d=1
aa={}
for t2 in prop[1].keys():
    aa[d]=t2
    d+=1    

In [2]:
def sl_fasta(n):# multiline fasta converted to single line fasta
    fasta=[]
    count=0
    with open(f'{n}','r') as inpt1:
        for j in inpt1:
            fasta.append(j.rstrip())
            if j.startswith('>'):
                count+=1
    acc_seq=[]
    c=0
    for k in range(len(fasta)):
        if fasta[k].startswith('>'):
            acc_seq.append(fasta[k]+'\n')
            join_=0
            for l in range(k+1,len(fasta)):
                if fasta[l].startswith('>') == False:
                    join_+=1
                else:
                    break
            acc_seq.append(''.join(fasta[k+1:k+1+join_])+'\n')
    return acc_seq

acc,seq,leng=[],[],[]
q=sl_fasta('course_cluster.txt')

for ele in q:
    if ele.startswith('>'):
        acc.append(ele.rstrip())
    else:
        if 'X' in ele:
            ele=ele.replace('X','') # removing X amino acid
        leng.append(len(ele.rstrip()))
        seq.append(ele.rstrip().upper())
print('The length of the smallest sequence:',min(leng))

ValueError: min() arg is an empty sequence

In [5]:
# nf normalizing frequency of occurence, 1=true
def rel_cal(data,v,pro):
    tem=[]
    for u in range(len(data)-v):
        te=[]
        for u1 in pro:
            x=((prop[u1][data[u]])-(prop[u1][data[u+v]]))**2
            te.append(x)
#             print(u1,u,u+v,data[u],data[u+v],x)
        tem.append(sum(te)/len(pro))
    return sum(tem)

def theta(data,lamb,pro):
    the={}
    for u in range(1,lamb+1):
        the[u]=(1/(len(data)-u))*rel_cal(data,u,pro)
    return the

def pse(data,lamb,w,pro,nf):
    thet=theta(data,lamb,pro)
    deno=1+(w*sum(thet.values()))
    p={}
    if nf==1:
        norm=(len(data))
    else:
        norm=1
    for u in range(1,21+lamb):
        if u>=1 and u<=20:
#             print(u,'natural')
            num=data.count(aa[u])/norm # frequency
#             print(f'{aa[u]}.......',(num/deno))
            p[u]=num/deno
        elif u>=21 and u<=20+lamb:
#             print(u,'pseudo')
            num=w*thet[u-20]
#             print(f'lambda_{u-20}.......',(num/deno))
            p[u]=num/deno
    return p

def collect(lamb,w,pro,nf):
    out=open(f'AAC_lyso.txt','w',encoding='utf-8')
    out.write('#')
    for ea in aa.values():
        out.write('\t'+ea)
    for eac in range(lamb):
        out.write('\t'+'\u03BB'+str(eac+1))
    out.write('\n')
    for e_seq in range(len(seq)):# single sequence taken for test
        q=pse(seq[e_seq],lamb,w,pro,nf)
        tem=acc[e_seq].split(' ')[0][1:]
        out.write(tem)
        for zx in range(len(q)):
            out.write('\t'+str(q[zx+1]))
        out.write('\n')
    out.close()
    print('Work done!!!')
collect(0,0.05,[60,2,3],1) 

Work done!!!


In [2]:
prop[1]

{'A': 0.6362505881446507,
 'C': 0.2976010815515301,
 'D': -0.9235895634357834,
 'E': -0.7593958632694219,
 'F': 1.2211906449873136,
 'G': 0.4925811004990844,
 'H': -0.4104842504159038,
 'I': 1.4161706639348677,
 'K': -1.539315939059639,
 'L': 1.087783263602145,
 'M': 0.6567748006654459,
 'N': -0.8004442883110122,
 'P': 0.12314527512477108,
 'Q': -0.8722790321337953,
 'R': -2.5963128838805907,
 'S': -0.18471791268715668,
 'T': -0.051310531301988004,
 'V': 1.1083074761229401,
 'W': 0.8312306070922051,
 'Y': 0.26681476277033733}